# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color="City",
    size="Humidity",
    frame_width=800,
    frame_height=600,
    title=f'City Data: Humidity and Location',
    fontsize={'title': '14pt'},
    line_color='black'
)

# Display the map
map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [35]:
# Narrow down cities that fit criteria and drop any results with null values
df = city_data_df[(city_data_df["Max Temp"] > 23) & (city_data_df["Max Temp"] < 32) & (city_data_df["Humidity"] <= 60) & (city_data_df["Cloudiness"] <= 50)]

# Drop any rows with null values
df = df.dropna()

# Display sample data
df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,trairi,-3.2778,-39.2689,31.07,51,34,8.09,BR,1666108038
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
73,73,nalut,30.3333,10.8500,25.41,27,0,5.11,LY,1666108275
98,98,itarema,-2.9248,-39.9167,31.07,54,14,9.48,BR,1666108293
106,106,palencia,42.4167,-4.5000,24.40,30,40,8.66,ES,1666108219


### Step 3: Create a new DataFrame called `hotel_df`.

In [36]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
12,trairi,BR,-3.2778,-39.2689,51,
63,banda,IN,25.4833,80.3333,52,
73,nalut,LY,30.3333,10.8500,27,
98,itarema,BR,-2.9248,-39.9167,54,
106,palencia,ES,42.4167,-4.5000,30,
152,kahului,US,20.8947,-156.4700,60,
239,rawson,AR,-43.3002,-65.1023,25,
240,tikaitnagar,IN,26.9500,81.5833,59,
253,tchaourou,BJ,8.8865,2.5975,56,
258,comodoro rivadavia,AR,-45.8667,-67.5000,24,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [37]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey" : geoapify_key,
    "categories" : "accommodation.hotel",
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make and API request using the params dictionary
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
trairi - nearest hotel: Pousada Villa Aurora
banda - nearest hotel: #acnindiafy21
nalut - nearest hotel: No hotel found
itarema - nearest hotel: Pousada Oásis
palencia - nearest hotel: Hotel Área de Servicio Los Chopos
kahului - nearest hotel: Maui Seaside Hotel
rawson - nearest hotel: Hotel Deportivo
tikaitnagar - nearest hotel: No hotel found
tchaourou - nearest hotel: Auberge
comodoro rivadavia - nearest hotel: No hotel found
aquiraz - nearest hotel: Pousada tia Fabíola
salalah - nearest hotel: Muscat International Hotel
santa rosalia - nearest hotel: Sol y Mar
saint-francois - nearest hotel: Chez Lily
kalemie - nearest hotel: Hotel du Lac
carutapera - nearest hotel: No hotel found
lakki marwat - nearest hotel: No hotel found
abu zabad - nearest hotel: No hotel found
brokopondo - nearest hotel: Matu eiland
tikrit - nearest hotel: فندق بلازا
lahij - nearest hotel: No hotel found
bilma - nearest hotel: No hotel found
pali - nearest hotel: Hotel Pawan Internationa

,City,Country,Lat,Lng,Humidity,Hotel Name
12,trairi,BR,-3.2778,-39.2689,51,Pousada Villa Aurora
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
73,nalut,LY,30.3333,10.8500,27,No hotel found
98,itarema,BR,-2.9248,-39.9167,54,Pousada Oásis
106,palencia,ES,42.4167,-4.5000,30,Hotel Área de Servicio Los Chopos
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
239,rawson,AR,-43.3002,-65.1023,25,Hotel Deportivo
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
253,tchaourou,BJ,8.8865,2.5975,56,Auberge
258,comodoro rivadavia,AR,-45.8667,-67.5000,24,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [38]:
%%capture --no-display

# Configure the map plot
map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color="City",
    line_color="black",
    size=80,
    frame_width=800,
    frame_height=600,
    hover_cols=["Hotel Name", "Country", "Humidity"],
    title=f'Best Vacation Spots: Humidity and Nearest Hotel Within 10km',
    fontsize={'title': '14pt'},
)

# Display the map
map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country,Humidity)